# Data Quality Tutorial - Python 1

## Prerequisities
You should have worked through the Data Quality Core training module which is a foundation for all data quality training. This will explain the Data Quality Dimensions (like Uniqueness), and how each measure is calculated.

For example, you should already be able to explain:
* the benefits of measuring data quality
* what each of the 6 DAMA Dimensions are and what they mean
* what a completeness score of 0.5 means
* how multiple scores are combined to give an overall data quality index

### Prior coding experience required
* You should know what pandas dataframes are
* You should have some basic python knowledge

## Aims of Python 1
* To be able to measure data quality on dataframes
* To use and understand all 8 data quality functions in the code
* To export the results to a CSV file


In [ ]:
from datetime import datetime, timedelta

import pandas as pd

# we import the rules, the data quality configuration and final report
from gchq_data_quality import (
    AccuracyRule,
    CompletenessRule,
    ConsistencyRule,
    DataQualityConfig,
    DataQualityReport,
    TimelinessRelativeRule,
    TimelinessStaticRule,
    UniquenessRule,
    ValidityNumericalRangeRule,
    ValidityRegexRule,
)

Let's create a simple dataframe with known data quality issues

In [ ]:
df = pd.DataFrame(
    {
        "id": [1, 2, 3, 3, 5],  # 4 /5 unique
        "name": ["John", "Jane", "Dave", None, "Missing"],  # 1 null value
        "age": [30, 25, 102, 15, -5],  # a negative age
        "email": [
            "john@example.com",
            "jane@example.com",
            "dave@example",
            "test@test.com",
            "alice@example.com",
        ],  # invalid 3rd email
        "category": ["A", "B", "C", "D", "X"],
        "score": [
            10,
            20,
            30,
            40,
            -1,
        ],  # missing scores are defined as -1
        "date": [
            datetime(2023, 1, 1),
            datetime(2023, 2, 1),
            datetime(2023, 3, 1),
            datetime(2021, 1, 1),  # one date too old
            datetime(2023, 5, 1),
        ],
    }
)

df.head()

We use a few custom classes to help enforce data validation on our data quality results, and provide some useful functionality which you will see later in Python 2, such as being able to generate a list of data quality rules you have applied (to save you typing them all out)

We use a package called Pydantic for these classes, which you may want to investigate (but this is not necessary for this notebook)

In [ ]:
FINAL_REPORT = DataQualityReport()

We can run a single rule by first defining it, then evaluating it against our dataframe
### Uniqueness

In [ ]:
uniqueness_rule = UniquenessRule(field="id")

We can view the details of this rule definition with to_dict() (for Pydantic users, this is equivalent to model_dump())
Where you can see the default values that have been assigned, such as:
* data_quality_dimensions - it's 'Uniqueness'
* rule_description - it's None as we haven't provided it

We evaluate the rule like this:

In [ ]:
dq_result = uniqueness_rule.evaluate(df)

In [ ]:
dq_result.model_dump(mode="json")

### Interpreting the output

#### Key data quality outputs
* The field - we looked at the 'id' column
* The pass rate - 0.8 (80%)
* The number of records we checked (records_evaluated) - 5
    * We can back-calculate the number of records that passed (0.8 * 5 = 4)
* The data quality dimension - Uniqueness

#### Diagnostic outputs
* The records failed sample will provide us values that failed
* The records failed IDs will tell us where they are in the dataframe (1st row is 0)

#### Additional observations
* You will notice that it has set some default values for you (such as the measurement_time, which defaults to 'now' in UTC)
* You may also notice that the rule_data has defaulted to a dictionary string containing all the parameters you entered, such that you can recreate this data quality rule from the output (covered in more detail in Python 2)

### Accessing values
You can access values as attributes of the result i.e. dq_result.name_of_variable (e.g. dq_result.pass_rate)


In [ ]:
print(
    rf" 4 \ 5 ids are actually unique, giving a data quality score of {dq_result.pass_rate}"
)
print(f"we can track the rows that are incorrect: {dq_result.records_failed_ids} ")
print(
    f"And show a sample of the values that failed the test : {dq_result.records_failed_sample}"
)

In [ ]:
# The records attribute of our DataQualityReport is a list, so we just append the score to it
FINAL_REPORT.results.append(dq_result)

FINAL_REPORT.to_dataframe()

### Completeness
Checks for the number of valid values

In [ ]:
completeness_rule = CompletenessRule(field="name")
completeness_result = completeness_rule.evaluate(df)
completeness_result.to_dict()

What if data has otherways of meaning a values is incomplete, like the word 'missing'?

In [ ]:
# all functions have na_values as a parameter, this can be a string (if one item) or a list
completeness_rule.na_values = "Missing"  # or ["Missing", "NA"]

# if 'Missing' signifies a NULL value in 'name' column, then we skip it
# our pass rate is therefore lower
completeness_result = completeness_rule.evaluate(df)
print(
    "Now we have added 'Missing' to be the same as NULL, our pass rate is lower:",
    completeness_result.pass_rate,
)

In [ ]:
FINAL_REPORT.results.append(completeness_result)
FINAL_REPORT.to_dataframe()

### Accuracy
This is the hardest dimension to measure within the data itself as really it should be cross-checked with an authoritative source
E.g. checking someone's date of birth with the actual birth certificate

For this package, we do an accuracy check by verifying the values in a column are drawn from an authoritative list

In [ ]:
accuracy_rule = AccuracyRule(field="category", valid_values=["A", "B", "C", "D"])
accuracy_result = accuracy_rule.evaluate(df)
accuracy_result.to_dict()["records_failed_sample"]

# here 'X' must be inaccurate as it is not one of the allowed values of A,B,C,D

If we had values we DIDN'T want to be present, we would set inverse to be 'True'

In [ ]:
# We will get the opposite result now (1/5 values will pass)
accuracy_rule.inverse = True
accuracy_rule.evaluate(df).to_dict()["records_failed_sample"]

### Ignoring NULL values.

It's more insightful to compare rules together with completeness data, so rules like accuracy by default will
ignore NULL values. As the question we are asking is: of all the records we are checking (that have values in them), which are accurate?

If we count NULL as 'inaccurate', then columns with large numbers of NULL values in them would dominate the pass rate.
To take an extreme example, a column with 1,000,000 rows, with most NULL, the pass rate will always be 0.0000 (to 4.d.p) irrespecitve of 
how accurate the small number of values actually are.

In [ ]:
print(
    f"We evaluate all the records that are not null: {accuracy_result.records_evaluated} records checked"
)
print("With 'X' being swapped for NULL..")
accuracy_rule.na_values = ["X"]
print(f"We evaluate fewer records: {accuracy_rule.evaluate(df).records_evaluated}")

In [ ]:
# You can bypass this behaviour, but we do not recommend it (other than in the ConsistencyRule), as it changes the meaning of the rule, and could
# cause confusion if comparing your data between teams
accuracy_rule.skip_if_null = "never"
accuracy_rule.evaluate(df).records_evaluated

In [ ]:
FINAL_REPORT.results.append(accuracy_result)
FINAL_REPORT.to_dataframe()

### Consistency
A logical internal check on the data, such as a date of birth being earlier than a date of death, or some other mathematical check on a single column.
Under the hood we use pandas 'eval' syntax which is quite flexible, this is the same syntax as used in pandas 'query' syntax e.g. df.query('my_column > 3')

#### BACKTICKS
You MUST use backticks (`) around each column name - this is how we extract the columns in our code for collecting suitable samples of output data. In pandas.eval(), backticks are only usually required if your column names have spaces in them, but in our package you must use them at all times.

#### Deciding on the 'field'
Even though consistency checks can operate across many columns at once, you still need to specify a 'field' parameter, pick the one that makes the most sense

#### Eval help pages
https://pandas.pydata.org/docs/reference/api/pandas.eval.html#pandas.eval


In [ ]:
# Simple expressions are a single statement
consistency_rule = ConsistencyRule(field="age", expression="`age` > 3")
consistency_result = consistency_rule.evaluate(df)
consistency_result.to_dict()

In [ ]:
# Complex expressions are a dictionary with keys of 'if' and 'then'
consistency_rule2 = ConsistencyRule(
    field="age", expression={"if": "`age` > 3", "then": "`score` <= 40"}
)
consistency_result2 = consistency_rule2.evaluate(df)
print("We have constrained our analysis using an 'if' in the expression.")
print(
    f"We are evaluating fewer records (only those that meet the 'if' criteria): {consistency_result2.records_evaluated}"
)
print(
    f"Our pass rate is now higher, even though the expression age > 3 is the same: {consistency_result2.pass_rate=}"
)

#### Handling Nulls with `skip_if_null`

The `skip_if_null` parameter controls how null (missing) values are handled during consistency checks. You can then decide if one column in a comparison being NULL makes the record inconsitent or not (or should it be ignored / skipped)

- `"any"`: Skip (exclude) rows where **any** relevant column is null.
- `"all"`: Skip rows only if **all** relevant columns are null (default behaviour).
- `"never"`: Include all rows, even those with nulls; nulls are passed into the expression.

You can apply this value to any rule, although we recommend only using it with the ConsistencyRule.

You cannot override it in the Completeness rule (it makes no sense to skip null values in a rule that is counting NULLs!)

In [ ]:
# IF we set the -5 value to be NULL and skip it... all our records pass
consistency_rule.na_values = -5
print(
    f"By setting -5 age as NULL, all our records now pass: {consistency_rule.evaluate(df).pass_rate}"
)
print(
    f"Because we are only evaluating non-null values: {consistency_rule.evaluate(df).records_evaluated}"
)

In [ ]:
# By passing in 'never' we pass NULL into the expression (and it fails because NULL is not bigger than 3, so we get a lower DQ score)
consistency_rule_never_skip = (
    consistency_rule.model_copy()
)  # just to avoid modifying the original
consistency_rule_never_skip.skip_if_null = "never"
never_skip_result = consistency_rule_never_skip.evaluate(df)

print(
    f"We now evalute more rules:\n{never_skip_result.records_evaluated=}\nand so have a lower pass rate:\n{never_skip_result.pass_rate=} "
)


### Pandas Eval Syntax Examples


Below are increasingly complex examples of part of expressions you might put in the `expression` parameter for the `consistency` function, showing how to use the pandas eval syntax. Note that we can use mathematical and boolean operators, and access pandas series methods using .str (string methods), .dt (datetime methods) - of which there are many. Some of these might not be actual consistency expressions, per se, but they give you an idea of what is achievable. You can then combine this functionality to create quite complex expressions.

You final consistency expressions MUST result in a boolean output (true or false)

Use BACKTICKS (`) around ALL column names:

```python
"`Age Column` > 0"
```

---

### 1. **Basic numeric comparison**
```python
"`age` > 0"
```
Checks that all ages are positive numbers.

---

### 2. **Simple equality**
```python
"`category` == 'A'"
```
Ensures the `category` column is 'A'.

---

### 3. **String methods**
```python
"`name`.str.istitle()"
```
Confirms that names are written in title case.

---

### 4. **Date comparison**
```python
"`date` > '2022-01-01'"
```
Ensures all dates are after January 1st, 2022.

---

### 5. **Numeric relationship between columns**
```python
"`score` < `age`"
```
Score must always be less than the age.

---

### 6. **Multiple conditions using boolean operators**
```python
"(`age` >= 18) & (`category`.isin(['A', 'B', 'C']))"
```
Only adult records in certain categories.

---

### 7. **String and numeric combined**
```python
"`email`.str.contains('@example.com') & (`score` > 10)"
```
Email must be company domain, and score must be above 10.

---

### 8. **Conditional logic with a dict expression**
```python
{"if": "`age` < 18", "then": "`category` == 'D'"}
```
If age is less than 18, category must be 'D'.

---

### 9. **Datetime attribute with comparison**
```python
"`date`.dt.year == 2023"
```
All dates should be in the year 2023.

---

### 10. **Multiple columns & string method combined**
```python
"`name`.notnull() & `email`.str.endswith('@example.com') & (`score` >= 20)"
```
Name must not be null, email must end with '@example.com', and score is at least 20.

---

### 11. **Complex if-then logic referencing 3 columns**
This is the sort of example where using skip_if_null is useful, is a record inconsistent if score is NULL for example? Or should you ignore that (skip_if_null = 'Any')
```python
{
  "if": "(`score` < `age`) & `email`.str.contains('@example.com')",
  "then": "`name`.str.startswith('J')"
}
```
If score is less than age and email is a company email, the name must start with 'J'.

---

### Consistency Syntax Examples — Numeric Distribution Focus

#### 12. **Column mean within a range**
```python
"abs(`score`.mean() - 50) < 5"
```
Checks if the mean score is within 5 points of 50.

---

#### 13. **Standard deviation not too high**
```python
"`score`.std() < 15"
```
Ensures the standard deviation of the `score` column is below 15.

---

#### 14. **No extreme outliers (values within 3 standard deviations)**
```python
'abs(`score` - `score`.mean()) <= (3* `score`.std())'
```
All scores must be within 3 standard deviations from the mean.

---

#### 15. **Median (50th percentile) check**
```python
"`score`.median() > 60"
```
The median score must be above 60.

---

#### 16. **Quantile-based (90% below threshold)**
```python
"`score`.quantile(0.9) < 90"
```
Ninety percent of scores must be below 90.

---




In [ ]:
FINAL_REPORT.results.append(consistency_result)
FINAL_REPORT.results.append(consistency_result2)
FINAL_REPORT.to_dataframe()

### Timeliness

#### Introduction
The aim of timeliness as a measure is to check a date falls within a certain range (i.e. later than 'start' and earlier than 'end').

We have two flavours of this function:
* timeliness_static (fixed start and end dates)
* timeliness_relative (relative start and end dates)

Requiring this flexibilty of date comparisons was one of the main reasons for writing our own package. It's not so simple as finding one date is earlier than another, we often require a date to fall within a certain window.

#### A note on 'time'
There are plenty of ways to get time calculations wrong, given time zones and British Summer Time.
Within the function we normalise everything to UTC. If the dates you are measuring are not written in ISO 8601 standard with timezone information or a UTC offset, then we assume it's UTC and raise a warning. 

If none of your times have timezone information then this won't matter (as everything will just change to UTC), but it does matter when stamping values like 'datetime.now()' which will return local time.

For best results, pre-convert your date values to UTC, and if using 'now' as a reference point use datetime.now(timezone.utc).

#### Timeliness Static
Used with an absolute (static) start and end date. The comparison is <= and >= (it's inclusive of start and end date)

In [ ]:
# we have one value earlier than 2023-01-01 (score should be 0.8)
# note how you can pass in a datetime string or a datetime object

timeliness_static_rule = TimelinessStaticRule(
    field="date", start_date="2023-01-01", end_date=datetime(2023, 6, 1)
)
timeliness_static_result = timeliness_static_rule.evaluate(df)
timeliness_static_result.to_dict()

Note how we have convereted start_date (and end_date) to timezone aware values in UTC


In [ ]:
timeliness_static_rule.start_date


##### Leaving start or end date as None
If start_date is None, then we don't check against it, we just check the date is no later than end_date

If end_date is None, we just check the date is no earlier than start_date.


In [ ]:
# We just check times are no later than 1 June 2023, so this will pass the date in 2021 (all records now pass)
timeliness_static_rule.start_date = None
timeliness_static_rule.evaluate(df).pass_rate

In [ ]:
FINAL_REPORT.results.append(timeliness_static_result)
FINAL_REPORT.to_dataframe()

#### Timeliness Relative
This is when you want a more dynamic check. You want to know your date is within a start and end period, relative to another date (a reference date). For example, you might want to ensure the booking date is in the future relative to the order date.

We allow for the reference date being 'now' (the default, in UTC of course) or a datetime object. Alternatively you can specify a reference_column - this will then compare each row of your dateframe and use the date values in that column as the reference.

We use timedeltas to define the window relative to the reference date. Negative means past, positive means future.

So setting start_timedelta='-5d' and end_timedelta='+6h' means your date should be within a time window of 5 days before and 6 hours after your reference date.

We stick to pandas timedelta formatting, so you can pass in 
* timedelta objects (timedelta(days=5))
* a compatible string ('5d')
* an ISO 8601 time duration string ('P5D') - just not using year (Y) or month (M) markers

##### Valid timedelta periods
Note that 'month' and 'year' are ambiguous due to leap years and variable numbers of days in months, so these are not valid for time durations

In [ ]:
# Use '0d' as a start_timedelta amount to mean 'equal to or later than the reference_date'
# Noting that if you don't specify a time in your reference date, then the default time of 00:00hrs will be used

# Here we are checking if the 'date' is within 2 years of the future of reference_date (1st Jan) - one of our valies was 2021, so we should have 1 failing record out of 5, (score of 0.8)

timeliness_relative_rule = TimelinessRelativeRule(
    field="date",
    reference_date="2023-01-01",
    start_timedelta=0,
    end_timedelta=timedelta(days=365 * 2),
)
timelienss_relative_result = timeliness_relative_rule.evaluate(df)
timelienss_relative_result.to_dict()

Let's pretend we have a column containing dates we want to compare on a per-row basis

In [ ]:
df["reference_date_column"] = df["date"] + timedelta(days=3)

Just be careful on the logic...as we are comparing 'date' to a reference date that is now 3 days in the future for each date,
our dates are all in the past relative to the reference column dates.

In [ ]:
timeliness_relative_rule2 = TimelinessRelativeRule(
    field="date",
    reference_column="reference_date_column",
    start_timedelta="-3d",
    end_timedelta="0d",
)
timeliness_relative_result2 = timeliness_relative_rule2.evaluate(df)
timeliness_relative_result2.pass_rate

In [ ]:
FINAL_REPORT.results.append(timelienss_relative_result)
FINAL_REPORT.results.append(timeliness_relative_result2)

### Validity

We use validity to mean one of two things:
* matching a Regular Expression (regex) pattern (e.g. just numbers [0-9]+)
    * if you are unfamiliar with regular expressions, there is plenty of support online (YouTube tutorials, regex calculators etc)
* falling within a numerical range (e.g. age should be >=1 and <=120)

We skip over any NULL values. This is a deliberate design choice, as you can use a validity measure in conjunction with a completeness measure to get a full picture of your data. If we said that NULL values were invalid (or valid) we'd distort the picture, it would be difficult to work out why your data was scoring the way it was.

As an example: imagine you have 1 million email records, and they are all null apart from 10 (but those 10 are all valid email addresses). If you included NULL in the valididy meausure, then the score would be 0.0000, as most records are NULL.
However, comparing with a compeleteness score, we can interpret, that 1) we hardly have any email records but 2) of those we have, they are all valid.

#### Validity Regex
We pass in a regular expression and check the % of records that matches.

In [ ]:
validity_regex_rule = ValidityRegexRule(
    field="email", regex_pattern=r"^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$"
)
validity_regex_result = validity_regex_rule.evaluate(df)
validity_regex_result.to_dict()

#### Validity Numerical Range

The check is inclusive (greater or equal to min_value, less than or equal to max_value)

If you leave one as None it will get replaced by negative infinity (min_value) or infinity (max_value)

In [ ]:
validity_numerical_range_rule = ValidityNumericalRangeRule(
    field="age", min_value=1, max_value=120
)
validity_numerical_range_result = validity_numerical_range_rule.evaluate(df)
validity_numerical_range_result.to_dict()

In [ ]:
rule_no_min = ValidityNumericalRangeRule(field="age", max_value=150)
rule_no_min.min_value

In [ ]:
# ignoring min_value - so negative ages will count as valid
rule_no_min.evaluate(df).pass_rate

In [ ]:
FINAL_REPORT.results.extend([validity_regex_result, validity_numerical_range_result])
FINAL_REPORT.to_dataframe()

## Final Tweaks to your Rules
Now you have your final Data Quality Report! Congratulations on getting this far. 

### Additional details on your rules
* The rule outputs a rule_data field - this contains all the information required to reconstruct the rule
    *... more on that magic feature in the Python 2 tutorial
* You can provide a rule ID (if you are logging or referencing your rules somewhere - this has no functionality in the code, it's just there in case you need it)

### Add details of what you are measuring

It would be useful to note down, within each rule, various things about the data you are measuring - the definitions of these terms were covered in the Data Quality Core PowerPoint:
* measurment_time (this defaults to 'now', but you can overwrite it)
* dataset name
* dataset ID
* measurement_sample
* lifecycle stage
    * make this sort alphabetically (if you take multiple measurements), so when you plot the data in a dashboard you can sort the data quality measures logically as they progress along the data lifecycle (e.g. 01 - ingest, 02 - enrich)

This information is contained in a DataQualityConfig object, which can contain a list of all of your rules

## Data Quality Config
To conduct an evaluation across many rules at a point in time, you should define a DataQualityConfig object - this captures the above details (dataset_name, dataset_id etc.)

A configuration is *executed* against a data source


In [ ]:
dq_config = DataQualityConfig(
    dataset_name="Tutorial Data",
    lifecycle_stage="02 Post Processing",
    measurement_time="2025-01-01",
    rules=[
        uniqueness_rule,
        completeness_rule,
        accuracy_rule,
        timeliness_relative_rule,
        timeliness_static_rule,
        consistency_rule,
        validity_numerical_range_rule,
        validity_regex_rule,
    ],
)

dq_report = dq_config.execute(df)
dq_report.to_dataframe()

## Exporting your final report
You can export the report to a dataframe / dictionary / JSON and then 
use existing pandas functionality to save to a CSV or Excel file.

You can modify the appearance of your dataframe (see the next cell)

In [ ]:
# you can how export your report to a dataframe and share the results as a CSV file
# We can modify the measurement_time format and adjust the decimal place accuracy (to make it more readable)
# If you want to align the invalid row numbers with the row number in Excel, you can shift it by 2. (Excel data starts at 2, not 0)
df_report = dq_report.to_dataframe(
    decimals=2, measurement_time_format="%Y-%m-%d %H:%M", records_failed_ids_shift=2
)
df_report.head()

In [ ]:
df_report.to_csv("resources/you_first_data_quality_report.csv", index=False)

## What Next?
You might be thinking: 
this is great, but:
* I don't want to have to write a separate function call for every rule I want to run
* I don't want to manually add each rule to a Data Quality Config
* I want to manage my regex patterns centrally, rather than write them out each time
* I want to specify my rules in a text file so it's easier to change and manage
    * and then run them all in one go
* I want an easy way of generating that rules text file, based off of my Data Quality Report
    * basically, 'please write out the rules that created this report'

This will be covered in our Python 2 module :-)